In [34]:
from __future__ import print_function

import tensorflow as tf
import os

# Dataset Parameters - CHANGE HERE
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
DATASET_PATH = 'E:\HolisticClassification\Dataset' # the dataset file or root folder path.
TEST_DATASET_PATH = 'E:\HolisticClassification\Dataset\Test'
BATCH_SIZE = 32

# Image Parameters
N_CLASSES = 2 # CHANGE HERE, total number of classes
IMG_HEIGHT = 32 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 32 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale
from PIL import Image
learning_rate = 0.001
num_steps = 10000
display_step = 100

# Network Parameters
DROP_OUT = 0.75 # Dropout, probability to keep units

# Build the data input


In [35]:
def read_images(dataset_path = DATASET_PATH, mode = MODE, batch_size = BATCH_SIZE):
    imagepaths, labels = list(), list()
    if mode == 'folder':
        label = 0
        classes = [f.path for f in os.scandir(DATASET_PATH) if f.is_dir() ] 
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.bmp') or sample.endswith('.BMP'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")

    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    # Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer([imagepaths, labels],
                                                 shuffle=True)

    # Read images from disk
    
    image = tf.image.decode_bmp(image, channels=CHANNELS)

    # Resize images to a common size
    image = tf.image.resize_images(image, [IMG_HEIGHT, IMG_WIDTH])

    # Normalize
    image = image * 1.0/127.5 - 1.0

    # Create batches
    X, Y = tf.train.batch([image, label], batch_size=batch_size, capacity=batch_size * 8, num_threads=4)
    return X, Y

In [31]:
X_Train, Y_Train = read_images()


In [9]:
X_Test, Y_Test = read_images(dataset_path = TEST_DATASET_PATH)
print(X_Test.shape)
print(Y_Test.shape)

(32, 32, 32, 3)
(32,)


In [37]:
def conv_net(x, reuse, is_training, n_classes = N_CLASSES, dropout = DROP_OUT):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out
    return out